In [37]:
%pip install simpy
%pip install "numpy<2" "fitter==1.7.1"
import simpy
import random
from fitter import Fitter
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
car_interarrival_data = [
    1.35166493e+00, 1.36167980e-01, 1.84202857e-01, 3.17178141e-01, 2.51452263e+00, 2.65475771e-02,
    1.67443339e-01, 2.09940735e+00, 3.41459398e+00, 1.49252804e-01, 8.74826243e-01, 5.09988862e-02,
    3.67731571e-01, 3.77030083e-01, 8.53583257e-02, 5.31342615e+00, 1.91600101e+00, 1.94930075e-01,
    2.04986668e+00, 2.85673099e+00, 9.75932581e-01, 1.94861521e-01, 6.87801682e-01, 6.63802815e-01,
    2.98096432e+00, 5.37112331e-01, 1.88873714e-01, 1.87810825e+00, 5.43372567e-01, 1.59540088e+00,
    1.35160483e+00, 1.26024903e+00, 3.26581711e-01, 1.90230924e+00, 4.93460497e-01, 1.74663821e-01,
    4.53647701e-01, 4.51542578e-04, 1.38575055e-01, 2.49413797e+00, 4.54583938e-01, 1.85416115e-01,
    3.09104530e-01, 1.59440932e-01, 5.30669164e-01, 1.31789452e+00, 1.63107760e+00, 1.95897103e-01,
    5.18152889e-01, 5.80059344e-01, 2.06985808e+00, 1.55236941e-01, 4.52139818e-01, 1.34950675e+00,
    8.34712429e-01, 1.48393461e-01, 7.90668730e-01, 1.09956407e+00, 2.13323567e-01, 9.73275625e-02,
    3.02919707e+00, 9.79011089e-01, 1.78671931e+00, 6.78908197e-01, 8.11495323e-01, 7.32587321e-01,
    2.12304209e+00, 3.62475404e-02, 4.90550291e-01, 3.93394704e-01, 2.25658935e+00, 8.56790200e-01,
    5.10164918e-01, 1.01885052e-01, 1.92661884e+00, 1.25556899e+00, 1.16785463e-01, 4.46713059e-01,
    1.03879301e+00, 6.22589722e-01, 6.70299754e-02, 1.40772612e+00, 1.43272098e+00, 4.77824425e-02,
    8.21307291e-01, 2.55688529e-02, 1.93705664e-01, 3.02083695e-01, 9.35247162e-01, 1.98320867e+00,
    2.01670473e-01, 7.48256391e-02, 2.13309391e+00, 5.43444933e-01, 6.30951003e-02, 1.63619863e-01,
    1.42022203e+00, 5.78601898e-01, 1.70263363e+00, 1.39347881e-02
]

In [39]:
# 최적의 도착 간격 분포 찾기
f = Fitter(car_interarrival_data, distributions=['uniform', 'norm', 'expon', 'gamma'])
f.fit()

best_distribution = f.get_best()
print("가장 적합한 분포:")
print(best_distribution)

가장 적합한 분포:
{'expon': {'loc': 0.000451542578, 'scale': 0.9064284160297801}}


2025-11-10 05:20:56.284 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted uniform distribution with error=11.54894)
2025-11-10 05:20:56.285 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted norm distribution with error=8.428303)
2025-11-10 05:20:56.295 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted expon distribution with error=4.093185)
2025-11-10 05:20:56.331 | INFO     | fitter.fitter:_fit_single_distribution:333 - Fitted gamma distribution with error=4.699831)


In [40]:
# 시뮬레이션 파라미터 설정
RANDOM_SEED = 7                # 난수 시드
NUM_MACHINES = 2               # 세차 기계의 수
WASHTIME = 1                   # 세차에 걸리는 시간 (단위: 시간)
SIM_TIME = 20                  # 총 시뮬레이션 시간 (단위: 시간)

random.seed(RANDOM_SEED)       # 난수 시드 설정
Q = []                         # 대기열 상태 저장
B = []                         # 자원 사용 상태 저장

In [41]:
# 세차 작업 정의
def wash(env, car):
    yield env.timeout(WASHTIME)  # 세차 시간 대기

# 고객 프로세스 정의
def car(env, name, carwash, num_machines):
    global Q, B # 그래프 그리기 위해서 적어놓은 것
    print(f'{name}이(가) {env.now:.2f}에 세차장에 도착했습니다.')  # 도착 시점 출력
    arrival_time = env.now # 나중에 대기시간 계산할 때 사용

    # 자원 요청
    req = carwash.request()            # request 객체 생성
    yield req  # 자원 요청 완료까지 대기

    # 대기 시간 기록
    Q.append((round(env.now, 2), len(carwash.queue))) # 대기열 길이 기록, 리스트 뒤에 값을 추가하겠다.
    B.append((round(env.now, 2), carwash.count)) # 자원 사용 상태 기록

    wait_time = env.now - arrival_time  # 대기 시간 계산
    if wait_time > 0:
        print(f'{name}이(가) {env.now:.2f}에 대기 후 세차장에 들어갔습니다.')  # 대기 후 입장
    else:
        print(f'{name}이(가) {env.now:.2f}에 바로 세차장에 들어갔습니다.')  # 바로 입장, <0이니까 시간은 0으로 환산됨.

    # 세차 작업 시작
    yield env.process(wash(env,name))                    # 총 시뮬레이션 시간을 sim_time으로 설정하여 실행  # 세차 프로세스 실행

    print(f'{name}이(가) {env.now:.2f}에 세차장을 떠났습니다.')  # 세차 완료 시점 출력

    # 서비스 완료 후 자원 반납
    carwash.release(req)
    # 대기 시간 기록
    Q.append((round(env.now, 2), len(carwash.queue)))
    B.append((round(env.now, 2), carwash.count))

# 세차장 초기화 및 고객 생성 설정
def setup(env, num_machines, scale):
    carwash = simpy.Resource(env, capacity=num_machines) # 세차 기계 자원 생성
    car_count = 1  # 자동차 번호 초기화

    # 초기 자동차 4대 생성
    for _ in range(4):
        env.process(car(env, f'Car {car_count}', carwash, num_machines))
        car_count += 1  # 자동차 번호 증가

    while True:
        # 지수 분포를 사용하여 도착 간격을 생성
        interarrival_time = random.expovariate(1/ scale)  # scale에 따라 평균 간격 조정, 1/scale 앞에 최적의 scale값을 넣어준거임
        yield env.timeout(interarrival_time)      # interarrival_time만큼 대기
        env.process(car(env, f"Customer {car_count}", carwash, num_machines))  # 고객 프로세스 생성
        car_count += 1  # 자동차 번호 증가

In [42]:
# 지수 분포에서의 scale 값 설정
scale = 0.9064284160297801

In [43]:
print('세차장 시작')
env = simpy.Environment()   # SimPy 환경 생성
env.process(setup(env, NUM_MACHINES, scale))     # 고객 생성 프로세스 추가
env.run(until = SIM_TIME)    # 시뮬레이션 실행

세차장 시작
Car 1이(가) 0.00에 세차장에 도착했습니다.
Car 2이(가) 0.00에 세차장에 도착했습니다.
Car 3이(가) 0.00에 세차장에 도착했습니다.
Car 4이(가) 0.00에 세차장에 도착했습니다.
Car 1이(가) 0.00에 바로 세차장에 들어갔습니다.
Car 2이(가) 0.00에 바로 세차장에 들어갔습니다.
Customer 5이(가) 0.35에 세차장에 도착했습니다.
Customer 6이(가) 0.50에 세차장에 도착했습니다.
Car 1이(가) 1.00에 세차장을 떠났습니다.
Car 2이(가) 1.00에 세차장을 떠났습니다.
Car 3이(가) 1.00에 대기 후 세차장에 들어갔습니다.
Car 4이(가) 1.00에 대기 후 세차장에 들어갔습니다.
Customer 7이(가) 1.46에 세차장에 도착했습니다.
Customer 8이(가) 1.53에 세차장에 도착했습니다.
Car 3이(가) 2.00에 세차장을 떠났습니다.
Car 4이(가) 2.00에 세차장을 떠났습니다.
Customer 5이(가) 2.00에 대기 후 세차장에 들어갔습니다.
Customer 6이(가) 2.00에 대기 후 세차장에 들어갔습니다.
Customer 9이(가) 2.22에 세차장에 도착했습니다.
Customer 10이(가) 2.63에 세차장에 도착했습니다.
Customer 11이(가) 2.69에 세차장에 도착했습니다.
Customer 5이(가) 3.00에 세차장을 떠났습니다.
Customer 6이(가) 3.00에 세차장을 떠났습니다.
Customer 7이(가) 3.00에 대기 후 세차장에 들어갔습니다.
Customer 8이(가) 3.00에 대기 후 세차장에 들어갔습니다.
Customer 12이(가) 3.33에 세차장에 도착했습니다.
Customer 13이(가) 3.36에 세차장에 도착했습니다.
Customer 14이(가) 3.88에 세차장에 도착했습니다.
Customer 15이(가) 3.95에 세차장에 도착했습니다.
Customer 7이(가) 4.00에 세차장을 떠났습니다

In [ ]:
# Q(t) 그래프
times, queue_lengths = zip(*Q)
xticks_times = sorted(set(times + (SIM_TIME,)))
plt.figure(figsize=(10, 5))
plt.step(times, queue_lengths, where='post', marker='o', linestyle='-', color='b')
plt.xlabel('Time')
plt.ylabel('Q')
plt.title('Q(t)')
plt.xticks(xticks_times, [f'{t:.2f}' for t in xticks_times])
plt.yticks(range(0, max(queue_lengths) + 2, 1))
plt.ylim(0, max(queue_lengths) + 1)
plt.xlim(0, SIM_TIME)
plt.grid(False)
plt.show()

In [ ]:
# B(t) 그래프
times_B, resource_states = zip(*B)
xticks_times_B = sorted(set(times_B + (SIM_TIME,)))
plt.figure(figsize=(10, 5))
plt.step(times_B, resource_states, where='post', marker='o', linestyle='-', color='r')
plt.xlabel('Time')
plt.ylabel('B')
plt.title('B(t)')
plt.xticks(xticks_times_B, [f'{t:.2f}' for t in xticks_times_B])
plt.yticks(range(0, NUM_MACHINES + 2, 1))
plt.ylim(0, NUM_MACHINES + 1)
plt.xlim(0, SIM_TIME)
plt.grid(False)
plt.show()

In [5]:
# 프로젝트 1 동질성 검사 
import pandas as pd
from scipy.stats import kruskal

# 엑셀 불러오기 (모든 시트 읽기)
xls = pd.ExcelFile('시뮬 데이터.xlsx')

# 전체 데이터 저장용 리스트
all_data = []

# 모든 시트를 순회하면서 1번, 2번 계산대 데이터를 추출
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    
    # 1번 계산대
    df1 = df.iloc[:, 0:7]  # 왼쪽 영역
    df1.columns = ["순번", "대기열도착시간", "서비스시작시간", "서비스종료시간", "대기시간", "서비스시간", "특이사항"]
    df1["계산대"] = "1번"
    
    # 2번 계산대
    df2 = df.iloc[:, 8:15]  # 오른쪽 영역
    df2.columns = ["순번", "대기열도착시간", "서비스시작시간", "서비스종료시간", "대기시간", "서비스시간", "특이사항"]
    df2["계산대"] = "2번"
    
    all_data.append(pd.concat([df1, df2]))

# 전체 합치기
data = pd.concat(all_data, ignore_index=True)

# 시간 형식을 초 단위로 변환 (예: 0:01:15 → 75초)
def time_to_seconds(t):
    if pd.isnull(t):
        return None
    try:
        h, m, s = map(int, str(t).split(':'))
        return h*3600 + m*60 + s
    except:
        return None

data['대기시간_sec'] = data['대기시간'].apply(time_to_seconds)
data['서비스시간_sec'] = data['서비스시간'].apply(time_to_seconds)

# 각 계산대별로 Kruskal–Wallis 검정
groups = [group['대기시간_sec'].dropna().values for name, group in data.groupby('계산대')]
stat, p = kruskal(*groups)

print(f"대기시간 → H = {stat:.3f}, p = {p:.4f}")
if p < 0.05:
    print("→ 계산대별 대기시간 분포가 다릅니다 (동질성 없음)")
else:
    print("→ 계산대별 대기시간 분포가 같습니다 (동질성 있음)")

# 서비스시간도 동일하게 검정
groups = [group['서비스시간_sec'].dropna().values for name, group in data.groupby('계산대')]
stat, p = kruskal(*groups)

print(f"서비스시간 → H = {stat:.3f}, p = {p:.4f}")
if p < 0.05:
    print("→ 계산대별 서비스시간 분포가 다릅니다 (동질성 없음)")
else:
    print("→ 계산대별 서비스시간 분포가 같습니다 (동질성 있음)")


대기시간 → H = 3.349, p = 0.0672
→ 계산대별 대기시간 분포가 같습니다 (동질성 있음)
서비스시간 → H = 2.581, p = 0.1082
→ 계산대별 서비스시간 분포가 같습니다 (동질성 있음)


In [6]:
# 전체 데이터프레임 크기 (행, 열)
print("전체 shape:", data.shape)

# 상위 5개 행 미리보기
print(data.head())

# 요약(열별 데이터 타입 / 결측치 개수 등)
print(data.info())

# 열별 기본 통계(수치형)
print(data.describe())

전체 shape: (620, 10)
    순번   대기열도착시간   서비스시작시간   서비스종료시간      대기시간     서비스시간  특이사항 계산대  대기시간_sec  \
0  순번    대기열도착시간   서비스시작시간   서비스종료시간      대기시간     서비스시간  특이사항  1번       NaN   
1    1  14:32:36  14:33:47  14:35:02  00:01:11  00:01:15   NaN  1번      71.0   
2    2  14:35:11  14:35:42  14:36:55  00:00:31  00:01:13   NaN  1번      31.0   
3    3  14:37:31  14:37:31  14:38:31  00:00:00  00:01:00   NaN  1번       0.0   
4    4  14:37:59  14:38:31  14:39:09  00:00:32  00:00:38   NaN  1번      32.0   

   서비스시간_sec  
0        NaN  
1       75.0  
2       73.0  
3       60.0  
4       38.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620 entries, 0 to 619
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   순번         594 non-null    object 
 1   대기열도착시간    589 non-null    object 
 2   서비스시작시간    589 non-null    object 
 3   서비스종료시간    589 non-null    object 
 4   대기시간       587 non-null    object 
 5   서비스시간      587 non-

In [7]:
# xls = pd.ExcelFile(file_path) 로 불러온 후
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet)
    print(f"시트: {sheet}, 행수: {len(df)}, 열수: {df.shape[1]}")

시트: 표2, 행수: 310, 열수: 19


In [12]:
import pandas as pd

df = pd.read_excel("시뮬 데이터.xlsx")

# 실제 컬럼명 확인
print(df.columns.tolist())

['10/06(월) - 1번 계산대', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', '10/06(월) - 2번 계산대', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18']


In [9]:
import pandas as pd

df = pd.read_excel("시뮬 데이터.xlsx")

# 실제 한글 열 이름을 그대로 써야 함
counter1_data = df["순번 1"].dropna().tolist()
counter2_data = df["순번 2"].dropna().tolist()

print("counter1_data = [")
for i, val in enumerate(counter1_data):
    end = ",\n" if (i + 1) % 10 == 0 else ", "
    print(f"    {val:.8e}{end}", end="")
print("]\n")

print("counter2_data = [")
for i, val in enumerate(counter2_data):
    end = ",\n" if (i + 1) % 10 == 0 else ", "
    print(f"    {val:.8e}{end}", end="")
print("]")


KeyError: '순번 1'

In [ ]:
# 10월 15일(수) 1번 계산대(85개)

from datetime import datetime, timedelta

# ---------------------------------------
# 대기시간, 서비스시간, 서비스종료시간 입력
# ---------------------------------------
wait_times = [
    '00:00:00','00:00:00','00:01:04','00:02:11','00:00:46','00:01:48','00:01:44','00:02:18','00:02:30','00:00:40',
    '00:00:00','00:00:17','00:00:30','00:00:35','00:01:28','00:00:29','00:01:33','00:01:45','00:02:28','00:01:55',
    '00:00:33','00:01:53','00:02:29','00:03:04','00:02:10','00:01:29','00:00:00','00:00:57','00:01:21','00:01:59',
    '00:01:41','00:01:27','00:00:42','00:00:33','00:02:07','00:03:22','00:03:51','00:03:33','00:03:21','00:03:52',
    '00:04:02','00:04:51','00:06:03','00:04:56','00:04:45','00:05:11','00:05:11','00:02:52','00:03:21','00:03:03',
    '00:03:30','00:03:47','00:03:43','00:04:34','00:04:59','00:04:51','00:05:26','00:05:00','00:05:57','00:06:25',
    '00:06:55','00:07:01','00:04:10','00:04:49','00:05:03','00:02:14','00:01:37','00:01:56','00:01:44','00:03:34',
    '00:00:12','00:00:00','00:00:00','00:01:11','00:01:41','00:02:08','00:01:44','00:00:58','00:01:03','00:00:47',
    '00:01:00','00:01:24','00:00:00','00:00:00','00:00:27'
]

service_times = [
    '00:00:18','00:01:06','00:02:20','00:00:27','00:02:11','00:00:17','00:00:35','00:01:06','00:00:21','00:00:31',
    '00:01:01','00:00:43','00:00:26','00:01:53','00:00:58','00:01:16','00:00:54','00:01:31','00:00:36','00:00:52',
    '00:01:27','00:01:11','00:00:43','00:00:48','00:00:29','00:00:23','00:01:13','00:00:40','00:00:59','00:00:21',
    '00:00:18','00:00:57','00:01:25','00:01:35','00:01:18','00:00:56','00:00:45','00:00:52','00:00:36','00:00:42',
    '00:01:27','00:01:25','00:00:26','00:01:08','00:00:29','00:00:34','00:00:56','00:00:51','00:00:51','00:01:08',
    '00:00:54','00:00:47','00:00:53','00:00:58','00:02:00','00:01:54','00:02:06','00:01:04','00:00:49','00:01:15',
    '00:00:32','00:00:36','00:01:29','00:02:04','00:00:31','00:01:04','00:00:55','00:01:01','00:02:27','00:00:28',
    '00:01:35','00:00:47','00:01:56','00:01:02','00:01:25','00:00:51','00:00:59','00:00:23','00:00:51','00:00:24',
    '00:00:37','00:00:24','00:01:08','00:00:53','00:00:51'
]

service_end_times = [
    '15:00:26','15:02:42','15:05:02','15:05:29','15:07:40','15:07:57','15:08:32','15:09:38','15:09:59','15:10:30',
    '15:11:37','15:12:20','15:12:46','15:14:39','15:15:37','15:16:53','15:17:47','15:19:18','15:19:54','15:20:46',
    '15:22:13','15:23:24','15:24:07','15:24:55','15:25:24','15:25:47','15:27:24','15:28:04','15:29:03','15:29:24',
    '15:29:42','15:30:39','15:32:04','15:33:39','15:34:57','15:35:53','15:36:38','15:37:30','15:38:06','15:38:48',
    '15:40:15','15:41:40','15:42:06','15:43:13','15:43:42','15:44:16','15:45:12','15:46:03','15:46:54','15:48:02',
    '15:48:56','15:49:43','15:50:36','15:51:34','15:53:34','15:55:28','15:57:34','15:58:38','15:59:27','16:00:42',
    '16:01:14','16:01:50','16:03:19','16:05:23','16:05:54','16:06:58','16:07:53','16:08:54','16:11:21','16:11:49',
    '16:13:24','16:15:11','16:17:09','16:18:11','16:19:36','16:20:27','16:21:26','16:21:49','16:22:50','16:23:14',
    '16:23:51','16:24:15','16:26:28','16:29:25','16:30:16'
]

# 10월 15일 (수) 2번 계산대 (76개)


wait_times = [
    '00:01:47','00:00:45','00:00:11','00:00:26','00:01:10','00:02:26','00:02:15','00:02:35','00:01:22','00:01:04',
    '00:01:00','00:00:31','00:00:38','00:00:56','00:03:06','00:02:57','00:03:09','00:03:05','00:01:30','00:01:54',
    '00:02:40','00:01:47','00:01:44','00:01:24','00:01:19','00:01:01','00:01:49','00:02:12','00:02:22','00:02:21',
    '00:03:07','00:03:48','00:03:59','00:04:21','00:04:10','00:03:22','00:03:02','00:03:33','00:04:04','00:04:33',
    '00:03:36','00:02:13','00:04:06','00:03:20','00:03:41','00:04:15','00:07:33','00:06:50','00:05:47','00:06:10',
    '00:05:01','00:05:10','00:03:08','00:02:30','00:01:57','00:01:48','00:00:42','00:01:15','00:01:45','00:01:11',
    '00:02:37','00:00:30','00:00:34','00:00:37','00:01:09','00:01:15','00:00:51','00:01:02','00:01:03','00:01:15',
    '00:01:12','00:01:25','00:01:07','00:01:36','00:00:00','00:00:18'
]

service_times = [
    '00:00:18','00:00:30','00:00:28','00:01:07','00:01:07','00:06:05','00:00:34','00:00:38','00:01:09','00:00:39',
    '00:00:49','00:00:36','00:01:02','00:02:24','00:00:57','00:00:30','00:00:12','00:00:14','00:01:44','00:01:02',
    '00:00:53','00:00:19','00:00:29','00:00:57','00:00:33','00:00:45','00:00:16','00:00:58','00:00:59','00:01:07',
    '00:00:41','00:00:16','00:01:01','00:00:48','00:00:24','00:00:34','00:00:56','00:00:58','00:00:43','00:00:19',
    '00:00:14','00:02:30','00:00:14','00:00:34','00:00:50','00:03:32','00:00:57','00:00:27','00:00:06','00:00:43',
    '00:00:21','00:00:20','00:00:49','00:00:57','00:01:04','00:01:01','00:00:32','00:00:53','00:01:17','00:01:26',
    '00:00:41','00:00:41','00:00:13','00:01:07','00:00:49','00:00:04','00:01:33','00:01:05','00:01:32','00:00:19',
    '00:01:14','00:01:10','00:00:38','00:00:41','00:00:39','00:01:20'
]

service_end_times = [
    '15:03:00','15:03:45','15:04:15','15:05:40','15:06:57','15:13:47','15:14:22','15:15:02','15:16:20','15:17:01',
    '15:17:57','15:18:35','15:19:51','15:22:18','15:23:20','15:24:00','15:24:19','15:24:40','15:26:27','15:27:33',
    '15:28:43','15:29:16','15:29:55','15:30:57','15:31:59','15:32:50','15:33:20','15:34:45','15:35:47','15:37:01',
    '15:37:46','15:38:11','15:39:15','15:40:11','15:40:50','15:41:31','15:42:29','15:43:35','15:44:25','15:44:52',
    '15:45:08','15:47:50','15:48:31','15:49:13','15:50:15','15:53:49','15:54:49','15:55:19','15:55:30','15:57:33',
    '15:57:58','15:58:30','15:59:27','16:00:36','16:01:47','16:02:50','16:03:44','16:04:48','16:08:39','16:10:07',
    '16:10:59','16:11:47','16:15:31','16:16:40','16:17:38','16:17:44','16:19:25','16:20:40','16:22:16','16:22:40',
    '16:24:12','16:25:36','16:26:39','16:27:24','16:28:55','16:30:19'
]

# 10월 8일 (수) 1번 계산대(74개)


wait_times = [
    '00:00:00','00:00:20','00:00:00','00:00:17','00:00:25','00:01:03','00:00:25','00:00:00','00:00:00','00:00:14',
    '00:00:24','00:00:23','00:00:00','00:00:22','00:00:00','00:00:26','00:00:32','00:00:45','00:00:32','00:02:06',
    '00:02:15','00:00:27','00:00:59','00:00:24','00:00:52','00:00:41','00:01:21','00:03:03','00:01:42','00:01:14',
    '00:01:05','00:01:02','00:01:36','00:02:02','00:02:14','00:00:00','00:00:00','00:00:00','00:00:04','00:00:00',
    '00:00:53','00:00:00','00:00:06','00:00:00','00:00:33','00:01:19','00:01:34','00:00:15','00:00:31','00:00:00',
    '00:01:24','00:00:35','00:00:00','00:00:32','00:00:36','00:01:15','00:01:11','00:02:07','00:01:45','00:07:01',
    '00:07:41','00:01:52','00:00:00','00:00:17','00:00:47','00:01:13','00:00:52','00:00:00','00:00:14','00:00:00',
    '00:00:07','00:00:56','00:00:00','00:00:14'
]

service_times = [
    '00:01:55','00:00:55','00:01:12','00:01:10','00:01:10','00:00:48','00:00:20','00:00:24','00:01:10','00:01:34',
    '00:00:48','00:01:13','00:00:27','00:00:25','00:00:46','00:00:22','00:00:55','00:01:05','00:02:04','00:00:57',
    '00:01:07','00:01:02','00:00:18','00:01:14','00:01:26','00:01:13','00:01:19','00:00:54','00:00:43','00:00:33',
    '00:00:41','00:01:24','00:01:13','00:00:52','00:01:02','00:02:33','00:00:25','00:00:50','00:01:08','00:01:20',
    '00:00:37','00:00:53','00:02:32','00:01:15','00:00:47','00:01:10','00:00:50','00:00:45','00:00:23','00:01:25',
    '00:00:27','00:00:31','00:00:51','00:00:18','00:00:52','00:01:19','00:01:41','00:01:53','00:06:57','00:01:12',
    '00:01:42','00:00:44','00:00:44','00:00:59','00:01:04','00:01:20','00:00:30','00:00:37','00:00:46','00:01:00',
    '00:01:18','00:00:58','00:00:16','00:01:43'
]

service_end_times = [
    '14:30:25','14:31:20','14:32:42','14:33:52','14:35:02','14:35:50','14:36:10','14:36:45','14:38:30','14:40:04',
    '14:40:52','14:42:05','14:43:25','14:43:50','14:45:40','14:46:02','14:46:57','14:48:02','14:50:06','14:51:03',
    '14:52:10','14:53:12','14:53:30','14:54:44','14:56:10','14:57:23','14:58:42','14:59:36','15:00:19','15:00:52',
    '15:01:33','15:02:57','15:04:10','15:05:02','15:06:04','15:10:36','15:11:26','15:12:52','15:14:00','15:15:24',
    '15:16:01','15:17:04','15:19:36','15:22:07','15:22:54','15:24:04','15:24:54','15:25:39','15:26:02','15:27:35',
    '15:28:02','15:28:33','15:30:01','15:30:20','15:31:12','15:32:31','15:34:12','15:36:05','15:43:02','15:44:14',
    '15:45:55','15:46:39','15:47:55','15:48:54','15:49:58','15:51:18','15:51:48','15:54:00','15:54:46','15:56:07',
    '15:57:25','15:58:23','15:59:15','16:00:58'
]


# 10월 8일 (수) 2번 계산대

# 2번 계산대 데이터

wait_times = [
    '00:00:00','00:00:28','00:00:09','00:00:23','00:00:00','00:00:00','00:02:07','00:00:34','00:00:08','00:00:49',
    '00:00:43','00:01:16','00:00:00','00:00:51','00:00:14','00:00:26','00:00:00','00:00:00','00:00:15','00:00:33',
    '00:00:18','00:01:11','00:01:35','00:01:37','00:00:00','00:00:57','00:01:42','00:02:55','00:02:08','00:03:02',
    '00:02:37','00:01:02','00:00:50','00:01:03','00:01:04','00:00:00','00:00:09','00:00:11','00:00:00','00:00:00',
    '00:00:19','00:00:40','00:00:00','00:00:30','00:00:00','00:00:00','00:00:00','00:00:19','00:00:00','00:01:19',
    '00:02:08','00:01:04','00:01:14','00:01:14','00:02:05','00:01:58','00:02:23','00:02:21','00:02:11','00:02:19',
    '00:02:49','00:03:07','00:03:31','00:03:43','00:02:24','00:02:36','00:02:44','00:00:00','00:00:00','00:00:24',
    '00:00:19','00:00:11','00:00:00','00:00:00','00:00:19','00:00:38'
]

service_times = [
    '00:01:34','00:00:37','00:01:01','00:01:17','00:00:42','00:01:20','00:00:48','00:01:10','00:00:58','00:01:08',
    '00:00:48','00:01:34','00:01:12','00:00:55','00:00:35','00:00:51','00:01:25','00:01:05','00:01:00','00:01:14',
    '00:00:54','00:00:59','00:00:47','00:00:32','00:01:15','00:01:00','00:01:27','00:00:56','00:01:05','00:00:38',
    '00:00:54','00:00:41','00:00:45','00:00:56','00:01:38','00:01:25','00:01:12','00:02:24','00:00:57','00:01:12',
    '00:00:48','00:01:08','00:00:56','00:01:30','00:01:28','00:01:18','00:01:16','00:00:58','00:01:36','00:01:06',
    '00:00:45','00:01:03','00:01:15','00:00:55','00:01:39','00:00:57','00:00:58','00:01:04','00:01:16','00:00:56',
    '00:01:01','00:00:29','00:00:55','00:00:42','00:00:15','00:01:09','00:01:06','00:01:06','00:00:53','00:01:40',
    '00:01:23','00:00:48','00:01:30','00:01:06','00:01:42','00:00:57'
]

service_end_times = [
    '14:31:09','14:31:46','14:32:47','14:34:04','14:34:53','14:36:43','14:37:31','14:38:41','14:39:39','14:40:47',
    '14:41:35','14:43:09','14:44:48','14:45:43','14:46:18','14:47:09','14:48:44','14:49:55','14:50:55','14:52:09',
    '14:53:03','14:54:02','14:54:49','14:55:21','14:57:11','14:58:11','14:59:38','15:00:34','15:01:39','15:02:17',
    '15:03:11','15:03:52','15:04:37','15:05:33','15:07:11','15:11:19','15:12:31','15:14:55','15:16:13','15:18:31',
    '15:19:19','15:20:27','15:21:51','15:23:21','15:24:57','15:26:22','15:27:59','15:28:57','15:30:57','15:32:03',
    '15:32:48','15:33:51','15:35:06','15:36:01','15:37:40','15:38:37','15:39:35','15:40:39','15:41:55','15:42:51',
    '15:43:52','15:44:21','15:45:16','15:45:58','15:46:13','15:47:22','15:48:51','15:50:05','15:51:45','15:53:08',
    '15:53:56','15:56:27','15:58:12','15:59:54','16:00:51'
]






